In [5]:
import pandas as pd
import numpy as np
import pymysql
from sklearn.cluster import AgglomerativeClustering as AC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt

In [6]:
connection = pymysql.connect(host="us-cdbr-iron-east-03.cleardb.net",
              user="bb8507107bd736",
              password="89bc7c14",
              db="heroku_94b4589a90ac9bc")

df = pd.read_sql("SELECT * FROM final_pitching_table", connection)
df.head()

,IP,SO,ER,W,IP_Avg,SO_Avg,ER_Avg,W_Avg,BatAvg1,BatAvg2,...,BatAvg9,FieldAvg1,FieldAvg2,FieldAvg3,FieldAvg4,FieldAvg5,FieldAvg6,FieldAvg7,FieldAvg8,FieldAvg9
0,6.0,4.0,1.0,0.0,5.05,4.250000,6.242500,0.250000,0.279591,0.320348,...,0.159143,0.989956,1.000000,0.984636,0.989542,1.000000,0.954143,0.991176,0.944316,1.000000
1,6.0,4.0,2.0,0.0,5.70,5.333333,1.046667,0.666667,0.719200,0.404316,...,0.186857,1.000000,1.000000,0.994522,0.846087,0.975533,1.000000,0.997333,1.000000,1.000000
2,5.0,5.0,3.0,0.0,5.86,4.600000,3.674000,0.200000,0.719200,0.202087,...,0.315400,1.000000,1.000000,0.994522,0.846087,0.994783,1.000000,0.997333,0.992312,1.000000
3,6.0,4.0,4.0,1.0,6.22,3.600000,4.652000,0.600000,0.242346,0.279393,...,0.244731,0.971111,0.998185,0.994333,1.000000,1.000000,0.928964,1.000000,0.940227,0.976903
4,4.0,7.0,5.0,0.0,6.02,4.200000,3.168000,0.400000,0.248889,0.285929,...,0.247348,0.995217,1.000000,0.981500,1.000000,1.000000,0.989280,0.995000,1.000000,1.000000


In [7]:
df = df.dropna()

In [12]:
def regression(target_att, selected_atts, df):
    Y = df[target_att]
    X = df[selected_atts]

    reg = LinearRegression().fit(X, Y)
#     print('Score: ', reg.score(X, Y))
#     print('Coef: ', reg.coef_)
#     print('Intercept: ', reg.intercept_)

    return reg

In [13]:
target_att = 'IP'
selected_attrs = ['SO_Avg', 'IP_Avg', 'FieldAvg6'] 
reg = regression(target_att, selected_attrs, df)
df['IP_Predicted'] = reg.predict( df[ ['SO_Avg', 'IP_Avg', 'FieldAvg6'] ])


target_att = 'SO'
selected_attrs = ['SO_Avg', 'IP_Avg', 'ER_Avg'] 
reg = regression(target_att, selected_attrs, df)
df['SO_Predicted'] = reg.predict( df[ ['SO_Avg', 'IP_Avg', 'ER_Avg'] ])


target_att = 'ER'
selected_attrs = ['SO_Avg', 'SO_Predicted', 'W_Avg'] 
reg = regression(target_att, selected_attrs, df)
df['ER_Predicted'] = reg.predict( df[ ['SO_Avg', 'SO_Predicted', 'W_Avg'] ])


target_att = 'W'
selected_attrs = ['SO_Avg', 'IP_Avg', 'SO_Predicted', 'ER_Avg', 'IP_Predicted'] 
reg = regression(target_att, selected_attrs, df)
df['W_Predicted'] = reg.predict( df[ ['SO_Avg', 'IP_Avg', 'SO_Predicted', 'ER_Avg', 'IP_Predicted'] ])


df['FanDuel_Score_True'] = df['W']*6 - 3*df['ER'] + 3*df['SO'] + 3*df['IP']

df['FanDuel_Score_Predicted'] = df['W_Predicted']*6 - 3*df['ER_Predicted'] + 3*df['SO_Predicted'] + 3*df['IP_Predicted']


Get correlation of the dependent variables against every other variable.

In [14]:
score_df = df.filter(regex = 'Score')

In [69]:
def compare_to_random(df):
    
    df = df.reset_index(drop = True)
    
    rand_index = np.random.randint(low = 0, high = 10)
    
    true_scores = df['FanDuel_Score_True']
    
    rand_choice = true_scores[rand_index]
    
    our_index = df['FanDuel_Score_Predicted'].idxmax()
    
    our_choice = true_scores[our_index]
    
    return our_choice > rand_choice

def test_model_performance(n = 10000) :

    better_than_rand_count = 0
    for iteration in range(n):
        better_than_rand_count += compare_to_random(score_df.sample(10))
        
    return better_than_rand_count/n
    

In [74]:
test_model_performance()

0.5551

In [34]:
score_df.sample(10)

,FanDuel_Score_True,FanDuel_Score_Predicted
102,57.0,30.871898
399,12.3,27.106391
374,12.0,24.718250
455,-2.7,29.426508
295,30.0,19.444573
681,24.0,26.293833
493,51.0,28.607077
417,33.0,30.306165
233,36.0,30.393789
141,15.0,24.860321


In [35]:
df.corr()[['IP','SO','ER','W']]

,IP,SO,ER,W
IP,1.000000,0.420606,-0.436602,0.414715
SO,0.420606,1.000000,-0.240092,0.252918
ER,-0.436602,-0.240092,1.000000,-0.416229
W,0.414715,0.252918,-0.416229,1.000000
IP_Avg,0.254796,0.155030,0.028856,0.067351
SO_Avg,0.159163,0.362263,-0.102517,0.094667
ER_Avg,-0.057133,-0.094146,-0.011792,0.053238
W_Avg,0.035137,0.062354,0.071024,-0.044197
BatAvg1,0.005831,-0.033031,0.038847,-0.020045
BatAvg2,-0.015979,-0.044432,0.058191,-0.006202


In [76]:
scaler = MinMaxScaler()
batting_vars = df.filter(regex = 'Field')

batting_vars_scaled = pd.DataFrame(scaler.fit_transform(batting_vars), columns=batting_vars.columns)

N = 

bat_var_cluster = AC(n_clusters=N).fit(batting_vars_scaled)

df['AC_Labels'] = pd.Series(bat_var_cluster.labels_ , index = df.index)

In [77]:
for i in range(N):
    
    print('cluster '+str(i)+': ', np.mean(df[ df['AC_Labels'] == i ]['ER']), len(df[df['AC_Labels'] == i ]))

cluster 0:  2.49290780141844 282
cluster 1:  2.191011235955056 89
cluster 2:  2.730769230769231 130
cluster 3:  2.772727272727273 88
cluster 4:  2.471830985915493 142
cluster 5:  2.4761904761904763 42
cluster 6:  2.6206896551724137 29


In [63]:
df[ df['AC_Labels'] == 4]

,IP,SO,ER,W,IP_Avg,SO_Avg,ER_Avg,W_Avg,BatAvg1,BatAvg2,...,FieldAvg1,FieldAvg2,FieldAvg3,FieldAvg4,FieldAvg5,FieldAvg6,FieldAvg7,FieldAvg8,FieldAvg9,AC_Labels
22,7.0,6.0,1.0,1.0,NaN,NaN,NaN,NaN,0.307833,0.159000,...,0.993800,0.983842,0.993667,0.998100,0.965333,1.000000,0.976440,0.998000,0.976000,4
23,6.2,4.0,1.0,0.0,5.020,3.000000,4.570000,0.400000,0.307833,0.159000,...,0.993800,0.983842,0.993667,0.998100,0.965333,1.000000,0.976440,0.992800,0.976000,4
44,6.0,4.0,1.0,0.0,5.300,6.500000,3.350000,0.750000,0.307833,0.159000,...,1.000000,0.997231,0.978000,1.000000,1.000000,0.998000,0.995640,0.985286,0.941783,4
45,6.0,9.0,1.0,1.0,5.750,6.250000,4.720000,0.500000,0.159000,0.270286,...,0.978952,0.997231,0.978000,1.000000,1.000000,0.985286,1.000000,0.941783,0.998400,4
121,5.0,1.0,2.0,0.0,5.100,1.000000,3.380000,1.000000,0.225222,0.249187,...,1.000000,0.985962,0.931640,0.994652,0.971040,0.980263,0.967875,0.992389,0.667000,4
125,5.2,8.0,2.0,1.0,5.860,5.400000,2.974000,0.200000,0.225222,0.249187,...,1.000000,0.985962,0.931640,0.994652,0.971040,0.980263,0.967875,0.992389,1.000000,4
128,3.2,0.0,7.0,0.0,5.240,3.600000,4.012000,0.000000,0.225222,0.249187,...,1.000000,0.985962,0.931640,0.994652,0.971040,1.000000,0.967875,0.992389,1.000000,4
131,6.0,3.0,1.0,0.0,5.320,4.000000,3.580000,0.400000,0.225222,0.249187,...,1.000000,0.985962,0.931640,0.994652,0.971040,1.000000,0.967875,0.964833,1.000000,4
132,3.0,2.0,5.0,0.0,6.020,3.400000,3.970000,0.400000,0.249187,0.364125,...,0.995643,0.951389,0.997040,0.982083,1.000000,0.995577,0.997143,0.967700,1.000000,4
209,8.0,5.0,2.0,1.0,5.000,3.000000,5.480000,0.000000,0.259630,0.209833,...,0.975824,0.980105,1.000000,0.995913,0.983545,1.000000,0.837500,0.982720,1.000000,4
